# Single-cell analysis to characterize the cellular microenvironment in hepatoblastoma

This ipython notebook works with the data from Bondoc *et al.* (2021) **_"Identification of distinct tumor cell populations and key genetic mechanisms through single cell sequencing in hepatoblastoma"_** & MacParland *et al.* (2018) **_"Single cell RNA sequencing of human liver reveals distinct intrahepatic macrophage populations"_**.

Datasets were taken through the Human Cell Atlas (HCA) portal to develop a single-cell pipeline that allowed us to characterize all the cell types in both datasets as an experiment previous to adapt the pipeline for our particular analysis with Hepatocarcinoma samples (in process of collection).

For Bondoc *et al.* we decided to use 5 tumoral samples and 2 controls. While for MacParland *et al.* we took 5 healthy liver (control) samples. After that and previous to preprocess the data we clean the data to only work with the **number of genes (n_genes), gene ids and the number of cells (n_cells)**.

Before any process we load the necessary libraries. The Scanpy API grants access to all its platform tools. It utilizes scientific libraries like NumPy and SciPy, and employs Pandas for data import and as the basis for its data structures, including the AnnData module. For visualizations, we use Matplotlib and Seaborn.

### **Timing**

Each notebook takes a different amount of time to run its process. Approximately running from the first notebook to the sixth notebook takes an average time of 3.5 hours, plus the time it may take to perform manual cell labeling (notebook 5). 

### Special Acknowledgments 

This work could not be possible without the work of **Haber et al.**, 2018 (Epithelial cells in different intestinal regions by 10x Chromium), **Ernesto Paas**, 2023 (Example Workflow for Computational Oncology) & **Mark Sanborn**, 2022 (Single-cell analysis complete class) as a guide to develop this pipeline.

In [3]:
#For downloads
import gdown, os, gzip, shutil

# Basic data management and plotting
import pandas as pd
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import os

# Scanpy fundamentals
import anndata as ad
import scanpy as sc
import seaborn as sb

# sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(figsize=(6, 6))
import scvi

/home/jupyter-mdiaz/.local/lib/python3.9/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/jupyter-mdiaz/.local/lib/python3.9/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


# Disclaimer

Data from the MacParland et al. were downloaded as loom files (7.67 GB), not the full information file which is a 120 samples fastq file (241.07 GB). Before preprocessing the samples, we loaded them to look all the information they conserved for the analysis. 

We used scanpy to preserve the information we were interested in from each loom file and saved them with a h5ad format for further analysis. 

While for Bondoc et al. data, they published their full data in a fastq file format with 42 samples (235.73 GB), for this analysis we used their 10X matrix files with a lower number of samples; 2 healthy liver samples (control) and 5 tumor samples from Hepatoblastoma (2.63 GB). 

Finally, we give a simple name for each sample and store them as h5ad files. 

# Loading healthy liver controls

In [4]:
%cd /datos/home/jupyter-mdiaz/experimento_3/COMPLETO/loom_files/

/datos/home/jupyter-mdiaz/experimento_3/COMPLETO/loom_files


In [5]:
adata1 = sc.read_loom('02f207ae-217b-42ce-9733-c03e61541fcc.loom', var_names='gene_symbols')

/home/jupyter-mdiaz/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [6]:
adata1.var_names_make_unique()

In [7]:
sc.pp.filter_genes(adata1, min_cells =10)
sc.pp.filter_cells(adata1, min_genes =200)

In [8]:
adata1

AnnData object with n_obs × n_vars = 5059 × 18023
    obs: 'antisense_reads', 'cell_barcode_fraction_bases_above_30_mean', 'cell_barcode_fraction_bases_above_30_variance', 'cell_names', 'duplicate_reads', 'emptydrops_FDR', 'emptydrops_IsCell', 'emptydrops_Limited', 'emptydrops_LogProb', 'emptydrops_PValue', 'emptydrops_Total', 'fragments_per_molecule', 'fragments_with_single_read_evidence', 'genes_detected_multiple_observations', 'genomic_read_quality_mean', 'genomic_read_quality_variance', 'genomic_reads_fraction_bases_quality_above_30_mean', 'genomic_reads_fraction_bases_quality_above_30_variance', 'input_id', 'molecule_barcode_fraction_bases_above_30_mean', 'molecule_barcode_fraction_bases_above_30_variance', 'molecules_with_single_read_evidence', 'n_fragments', 'n_genes', 'n_mitochondrial_genes', 'n_mitochondrial_molecules', 'n_molecules', 'n_reads', 'noise_reads', 'pct_mitochondrial_molecules', 'perfect_cell_barcodes', 'perfect_molecule_barcodes', 'reads_mapped_exonic', 'reads_map

In [9]:
adata1.obs = adata1.obs[['n_genes']]
cols_to_keep = ['Gene', 'ensembl_ids', 'n_cells']
adata1.var = adata1.var[cols_to_keep]

adata1.var.rename(columns={'ensembl_ids': 'gene_ids'}, inplace=True)

adata1.var.set_index('Gene', inplace=True)
adata1.var.index.name = None

In [10]:
adata1.var_names_make_unique()

In [11]:
adata1

AnnData object with n_obs × n_vars = 5059 × 18023
    obs: 'n_genes'
    var: 'gene_ids', 'n_cells'

In [12]:
adata2 = sc.read_loom('2b965070-e2c5-4c26-92c9-c94483f1a00c.loom', var_names='gene_symbols')

/home/jupyter-mdiaz/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [13]:
adata2.var_names_make_unique()

In [14]:
sc.pp.filter_genes(adata2, min_cells =10)
sc.pp.filter_cells(adata2, min_genes =200)

In [15]:
adata2

AnnData object with n_obs × n_vars = 7767 × 16945
    obs: 'antisense_reads', 'cell_barcode_fraction_bases_above_30_mean', 'cell_barcode_fraction_bases_above_30_variance', 'cell_names', 'duplicate_reads', 'emptydrops_FDR', 'emptydrops_IsCell', 'emptydrops_Limited', 'emptydrops_LogProb', 'emptydrops_PValue', 'emptydrops_Total', 'fragments_per_molecule', 'fragments_with_single_read_evidence', 'genes_detected_multiple_observations', 'genomic_read_quality_mean', 'genomic_read_quality_variance', 'genomic_reads_fraction_bases_quality_above_30_mean', 'genomic_reads_fraction_bases_quality_above_30_variance', 'input_id', 'molecule_barcode_fraction_bases_above_30_mean', 'molecule_barcode_fraction_bases_above_30_variance', 'molecules_with_single_read_evidence', 'n_fragments', 'n_genes', 'n_mitochondrial_genes', 'n_mitochondrial_molecules', 'n_molecules', 'n_reads', 'noise_reads', 'pct_mitochondrial_molecules', 'perfect_cell_barcodes', 'perfect_molecule_barcodes', 'reads_mapped_exonic', 'reads_map

In [16]:
adata2.obs = adata2.obs[['n_genes']]
cols_to_keep = ['Gene', 'ensembl_ids', 'n_cells']
adata2.var = adata2.var[cols_to_keep]

adata2.var.rename(columns={'ensembl_ids': 'gene_ids'}, inplace=True)

adata2.var.set_index('Gene', inplace=True)
adata2.var.index.name = None

In [17]:
adata2.var_names_make_unique()

In [18]:
adata2

AnnData object with n_obs × n_vars = 7767 × 16945
    obs: 'n_genes'
    var: 'gene_ids', 'n_cells'

In [19]:
adata3 = sc.read_loom('3f9058b9-4243-4ef1-a345-857a5b9ff78e.loom', var_names='gene_symbols')

/home/jupyter-mdiaz/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [20]:
adata3.var_names_make_unique()

In [21]:
sc.pp.filter_genes(adata3, min_cells =10)
sc.pp.filter_cells(adata3, min_genes =200)

In [22]:
adata3

AnnData object with n_obs × n_vars = 2760 × 16425
    obs: 'antisense_reads', 'cell_barcode_fraction_bases_above_30_mean', 'cell_barcode_fraction_bases_above_30_variance', 'cell_names', 'duplicate_reads', 'emptydrops_FDR', 'emptydrops_IsCell', 'emptydrops_Limited', 'emptydrops_LogProb', 'emptydrops_PValue', 'emptydrops_Total', 'fragments_per_molecule', 'fragments_with_single_read_evidence', 'genes_detected_multiple_observations', 'genomic_read_quality_mean', 'genomic_read_quality_variance', 'genomic_reads_fraction_bases_quality_above_30_mean', 'genomic_reads_fraction_bases_quality_above_30_variance', 'input_id', 'molecule_barcode_fraction_bases_above_30_mean', 'molecule_barcode_fraction_bases_above_30_variance', 'molecules_with_single_read_evidence', 'n_fragments', 'n_genes', 'n_mitochondrial_genes', 'n_mitochondrial_molecules', 'n_molecules', 'n_reads', 'noise_reads', 'pct_mitochondrial_molecules', 'perfect_cell_barcodes', 'perfect_molecule_barcodes', 'reads_mapped_exonic', 'reads_map

In [23]:
adata3.obs = adata3.obs[['n_genes']]
cols_to_keep = ['Gene', 'ensembl_ids', 'n_cells']
adata3.var = adata3.var[cols_to_keep]

adata3.var.rename(columns={'ensembl_ids': 'gene_ids'}, inplace=True)

adata3.var.set_index('Gene', inplace=True)
adata3.var.index.name = None

In [24]:
adata3.var_names_make_unique()

In [25]:
adata3

AnnData object with n_obs × n_vars = 2760 × 16425
    obs: 'n_genes'
    var: 'gene_ids', 'n_cells'

In [26]:
adata4 = sc.read_loom('42998245-4c46-488e-ad04-14550074d6d4.loom', var_names='gene_symbols')

/home/jupyter-mdiaz/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [27]:
adata4.var_names_make_unique()

In [28]:
sc.pp.filter_genes(adata4, min_cells =10)
sc.pp.filter_cells(adata4, min_genes =200)

In [29]:
adata4

AnnData object with n_obs × n_vars = 3122 × 16322
    obs: 'antisense_reads', 'cell_barcode_fraction_bases_above_30_mean', 'cell_barcode_fraction_bases_above_30_variance', 'cell_names', 'duplicate_reads', 'emptydrops_FDR', 'emptydrops_IsCell', 'emptydrops_Limited', 'emptydrops_LogProb', 'emptydrops_PValue', 'emptydrops_Total', 'fragments_per_molecule', 'fragments_with_single_read_evidence', 'genes_detected_multiple_observations', 'genomic_read_quality_mean', 'genomic_read_quality_variance', 'genomic_reads_fraction_bases_quality_above_30_mean', 'genomic_reads_fraction_bases_quality_above_30_variance', 'input_id', 'molecule_barcode_fraction_bases_above_30_mean', 'molecule_barcode_fraction_bases_above_30_variance', 'molecules_with_single_read_evidence', 'n_fragments', 'n_genes', 'n_mitochondrial_genes', 'n_mitochondrial_molecules', 'n_molecules', 'n_reads', 'noise_reads', 'pct_mitochondrial_molecules', 'perfect_cell_barcodes', 'perfect_molecule_barcodes', 'reads_mapped_exonic', 'reads_map

In [30]:
adata4.obs = adata4.obs[['n_genes']]
cols_to_keep = ['Gene', 'ensembl_ids', 'n_cells']
adata4.var = adata4.var[cols_to_keep]

adata4.var.rename(columns={'ensembl_ids': 'gene_ids'}, inplace=True)

adata4.var.set_index('Gene', inplace=True)
adata4.var.index.name = None

In [31]:
adata3.var_names_make_unique()

In [32]:
adata4

AnnData object with n_obs × n_vars = 3122 × 16322
    obs: 'n_genes'
    var: 'gene_ids', 'n_cells'

In [38]:
adata5 = sc.read_loom('d1782f60-75d3-41a2-a6d5-8218daab2e31.loom', var_names='gene_symbols')

/home/jupyter-mdiaz/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [39]:
adata5.var_names_make_unique()

In [41]:
sc.pp.filter_genes(adata5, min_cells =10)
sc.pp.filter_cells(adata5, min_genes =200)

In [42]:
adata5

AnnData object with n_obs × n_vars = 8179 × 17235
    obs: 'antisense_reads', 'cell_barcode_fraction_bases_above_30_mean', 'cell_barcode_fraction_bases_above_30_variance', 'cell_names', 'duplicate_reads', 'emptydrops_FDR', 'emptydrops_IsCell', 'emptydrops_Limited', 'emptydrops_LogProb', 'emptydrops_PValue', 'emptydrops_Total', 'fragments_per_molecule', 'fragments_with_single_read_evidence', 'genes_detected_multiple_observations', 'genomic_read_quality_mean', 'genomic_read_quality_variance', 'genomic_reads_fraction_bases_quality_above_30_mean', 'genomic_reads_fraction_bases_quality_above_30_variance', 'input_id', 'molecule_barcode_fraction_bases_above_30_mean', 'molecule_barcode_fraction_bases_above_30_variance', 'molecules_with_single_read_evidence', 'n_fragments', 'n_genes', 'n_mitochondrial_genes', 'n_mitochondrial_molecules', 'n_molecules', 'n_reads', 'noise_reads', 'pct_mitochondrial_molecules', 'perfect_cell_barcodes', 'perfect_molecule_barcodes', 'reads_mapped_exonic', 'reads_map

In [43]:
adata5.obs = adata5.obs[['n_genes']]
cols_to_keep = ['Gene', 'ensembl_ids', 'n_cells']
adata5.var = adata5.var[cols_to_keep]

adata5.var.rename(columns={'ensembl_ids': 'gene_ids'}, inplace=True)

adata5.var.set_index('Gene', inplace=True)
adata5.var.index.name = None

In [44]:
adata5.var_names_make_unique()

In [45]:
adata5

AnnData object with n_obs × n_vars = 8179 × 17235
    obs: 'n_genes'
    var: 'gene_ids', 'n_cells'

# Loading Hepatoblastoma samples

In [46]:
%cd /datos/home/jupyter-mdiaz/experimento_3/COMPLETO/hb_filtered_files/

/datos/home/jupyter-mdiaz/experimento_3/COMPLETO/hb_filtered_files


In [47]:
adata6 = sc.read_10x_mtx('HB17_background_filtered_feature_bc_matrix', var_names='gene_symbols')

In [48]:
adata6.var_names_make_unique()

In [49]:
sc.pp.filter_genes(adata6, min_cells =10)
sc.pp.filter_cells(adata6, min_genes =200)

In [50]:
adata6.var = adata6.var.drop(columns=['feature_types'])
adata6.var_names_make_unique()
adata6

AnnData object with n_obs × n_vars = 11197 × 16895
    obs: 'n_genes'
    var: 'gene_ids', 'n_cells'

In [51]:
adata7 = sc.read_10x_mtx('HB53_background_filtered_feature_bc_matrix', var_names='gene_symbols')

In [52]:
adata7.var_names_make_unique()

In [53]:
sc.pp.filter_genes(adata7, min_cells =10)
sc.pp.filter_cells(adata7, min_genes =200)

In [54]:
adata7.var = adata7.var.drop(columns=['feature_types'])
adata7.var_names_make_unique()
adata7

AnnData object with n_obs × n_vars = 8540 × 22277
    obs: 'n_genes'
    var: 'gene_ids', 'n_cells'

In [55]:
adata8 = sc.read_10x_mtx('HB17_tumor_filtered_feature_bc_matrix', var_names='gene_symbols')

In [56]:
adata8.var_names_make_unique()

In [57]:
sc.pp.filter_genes(adata8, min_cells =10)
sc.pp.filter_cells(adata8, min_genes =200)

In [58]:
adata8.var = adata8.var.drop(columns=['feature_types'])
adata8.var_names_make_unique()
adata8

AnnData object with n_obs × n_vars = 7995 × 22112
    obs: 'n_genes'
    var: 'gene_ids', 'n_cells'

In [59]:
adata9 = sc.read_10x_mtx('HB30_tumor_filtered_feature_bc_matrix', var_names='gene_symbols')

In [60]:
adata9.var_names_make_unique()

In [61]:
sc.pp.filter_genes(adata9, min_cells =10)
sc.pp.filter_cells(adata9, min_genes =200)

In [62]:
adata9.var = adata9.var.drop(columns=['feature_types'])
adata9.var_names_make_unique()
adata9

AnnData object with n_obs × n_vars = 19042 × 24433
    obs: 'n_genes'
    var: 'gene_ids', 'n_cells'

In [63]:
adata10 = sc.read_10x_mtx('HB53_tumor_filtered_feature_bc_matrix', var_names='gene_symbols')

In [64]:
adata10.var_names_make_unique()

In [65]:
sc.pp.filter_genes(adata10, min_cells =10)
sc.pp.filter_cells(adata10, min_genes =200)

In [66]:
adata10.var = adata10.var.drop(columns=['feature_types'])
adata10.var_names_make_unique()
adata10

AnnData object with n_obs × n_vars = 12832 × 24324
    obs: 'n_genes'
    var: 'gene_ids', 'n_cells'

In [67]:
adata11 = sc.read_10x_mtx('HB17_PDX_filtered_feature_bc_matrix', var_names='gene_symbols')

In [68]:
adata11.var_names_make_unique()

In [69]:
sc.pp.filter_genes(adata11, min_cells =10)
sc.pp.filter_cells(adata11, min_genes =200)

In [70]:
adata11.var = adata11.var.drop(columns=['feature_types'])
adata11.var_names_make_unique()
adata11

AnnData object with n_obs × n_vars = 8027 × 22405
    obs: 'n_genes'
    var: 'gene_ids', 'n_cells'

In [71]:
adata12 = sc.read_10x_mtx('HB53_tumor_filtered_feature_bc_matrix', var_names='gene_symbols')

In [72]:
adata12.var_names_make_unique()

In [73]:
sc.pp.filter_genes(adata12, min_cells =10)
sc.pp.filter_cells(adata12, min_genes =200)

In [74]:
adata12.var = adata12.var.drop(columns=['feature_types'])
adata12.var_names_make_unique()
adata12

AnnData object with n_obs × n_vars = 12832 × 24324
    obs: 'n_genes'
    var: 'gene_ids', 'n_cells'

# Creating h5ad files 

In [75]:
%cd '/datos/home/jupyter-mdiaz/scRNAseq_pipeline/initial_h5ad'

/datos/home/jupyter-mdiaz/scRNAseq_pipeline/initial_h5ad


In [76]:
adata1.write_h5ad('control1')

In [77]:
adata2.write_h5ad('control2')

In [78]:
adata3.write_h5ad('control3')

In [79]:
adata4.write_h5ad('control4')

In [80]:
adata5.write_h5ad('control5')

In [81]:
adata6.write_h5ad('control6')

In [82]:
adata7.write_h5ad('control7')

In [83]:
adata8.write_h5ad('tumor_s1')

In [84]:
adata9.write_h5ad('tumor_s2')

In [85]:
adata10.write_h5ad('tumor_s3')

In [86]:
adata11.write_h5ad('tumor_s4')

In [87]:
adata12.write_h5ad('tumor_s5')